### Library Import

In [1]:
import os
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

### Data Load

In [2]:
# 파일 호출
data_path: str = "../../data"
#print(os.getcwd())
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)

In [3]:
# HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]

# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


100%|██████████| 107/107 [00:03<00:00, 29.00it/s]


### EDA (Explanatory Data Analysis) + Data Preprocessing

### 1차 클렌징

In [5]:
train_df = df.loc[df["_type"] == "train"]
# 각 열에서 누락된 값의 수를 계산
missing_values = train_df.isnull().sum()

# 누락된 값의 백분율 계산
missing_percentage = (missing_values / len(train_df)) * 100

# 누락된 값 비율을 기준으로 열 정렬
sorted_missing_percentage = missing_percentage.sort_values(ascending=False)
sorted_missing_percentage

hourly_market-data_liquidations_ftx_all_symbol_short_liquidations_usd     100.0
hourly_market-data_open-interest_bitmex_open_interest                     100.0
hourly_market-data_open-interest_all_exchange_open_interest               100.0
hourly_market-data_open-interest_gate_io_open_interest                    100.0
hourly_market-data_open-interest_deribit_open_interest                    100.0
                                                                          ...  
hourly_market-data_open-interest_htx_global_btc_usd_open_interest           0.0
hourly_network-data_transactions-count_transactions_count_total             0.0
target                                                                      0.0
ID                                                                          0.0
hourly_market-data_open-interest_all_exchange_all_symbol_open_interest      0.0
Length: 255, dtype: float64

전부 누락된 컬럼은 제외한 후,
남은 결측치는 이동평균으로 대체한다.

In [6]:
# missing_value의 비율이 100%가 아닌 column만 추출
non_missing_columns = sorted_missing_percentage[sorted_missing_percentage != 100.0].index.tolist()
non_missing_columns.remove('ID')
non_missing_columns.remove('target')
non_missing_columns.remove('_type')

new_data = train_df[['ID', 'target', '_type'] + non_missing_columns]
new_data

,ID,target,_type,hourly_market-data_open-interest_binance_btc_busd_open_interest,hourly_market-data_liquidations_binance_btc_busd_short_liquidations_usd,hourly_market-data_liquidations_binance_btc_busd_long_liquidations,hourly_market-data_liquidations_binance_btc_busd_short_liquidations,hourly_market-data_liquidations_binance_btc_busd_long_liquidations_usd,hourly_market-data_funding-rates_bybit_funding_rates,hourly_market-data_taker-buy-sell-stats_bybit_taker_sell_ratio,...,hourly_market-data_liquidations_binance_all_symbol_short_liquidations_usd,hourly_network-data_hashrate_hashrate,hourly_network-data_blockreward_blockreward,hourly_network-data_blockreward_blockreward_usd,hourly_network-data_block-count_block_count,hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close,hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_volume,hourly_market-data_open-interest_htx_global_btc_usd_open_interest,hourly_network-data_transactions-count_transactions_count_total,hourly_market-data_open-interest_all_exchange_all_symbol_open_interest
0,2023-01-01 00:00:00,2.0,train,3.180371e+08,0.0,0.0,0.0,0.0,0.010000,0.411867,...,0.00000,5.062917e+11,75.561037,1.248565e+06,12.0,16536.747967,5516.420322,3.412540e+07,11457.0,6.271344e+09
1,2023-01-01 01:00:00,1.0,train,3.183281e+08,0.0,0.0,0.0,0.0,0.010000,0.225491,...,11833.56104,1.687639e+11,25.256248,4.176322e+05,4.0,16557.136536,4513.341881,3.456750e+07,5832.0,6.288683e+09
2,2023-01-01 02:00:00,1.0,train,3.179729e+08,0.0,0.0,0.0,0.0,0.010000,0.635430,...,0.00000,3.375278e+11,50.312978,8.321735e+05,8.0,16548.149805,4310.904314,3.452090e+07,5550.0,6.286796e+09
3,2023-01-01 03:00:00,1.0,train,3.179643e+08,0.0,0.0,0.0,0.0,0.010000,0.412550,...,0.00000,2.109549e+11,31.469829,5.199754e+05,5.0,16533.632875,4893.417864,3.438940e+07,5245.0,6.284575e+09
4,2023-01-01 04:00:00,2.0,train,3.174773e+08,0.0,0.0,0.0,0.0,0.010000,0.537532,...,0.00000,2.953368e+11,44.094866,7.286038e+05,7.0,16524.712159,5209.002297,3.439200e+07,6942.0,6.291582e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00,1.0,train,NaN,NaN,NaN,NaN,NaN,0.024847,0.604128,...,600.62560,7.731601e+11,76.618081,3.265909e+06,9.0,42597.709522,1002.251559,4.921820e+07,41286.0,1.054230e+10
8756,2023-12-31 20:00:00,1.0,train,NaN,NaN,NaN,NaN,NaN,0.023523,0.400193,...,279693.16923,3.436267e+11,33.530340,1.427679e+06,4.0,42535.570787,1070.042124,4.916770e+07,21585.0,1.051484e+10
8757,2023-12-31 21:00:00,0.0,train,NaN,NaN,NaN,NaN,NaN,0.022368,0.560964,...,51776.92115,2.577200e+11,25.665014,1.093419e+06,3.0,42497.523419,1026.473547,4.914800e+07,13803.0,1.048598e+10
8758,2023-12-31 22:00:00,2.0,train,NaN,NaN,NaN,NaN,NaN,0.021547,0.724267,...,148758.33579,1.718134e+11,17.758057,7.537248e+05,2.0,42257.026218,1715.192245,4.919720e+07,8070.0,1.032844e+10


In [7]:
# 이동평균으로 결측치 대체
new_df_stab = new_data[non_missing_columns]

# train
window_size = 3
new_df_stab = new_df_stab.apply(lambda col: col.fillna(col.rolling(window=window_size, min_periods=1).mean()))
new_df_stab = new_df_stab.fillna(method='ffill').fillna(method='bfill')


# 결측치 처리한 new_df 정의
new_train_df = pd.concat([new_data[['ID','target','_type']], new_df_stab], axis=1)

/tmp/ipykernel_751711/2767777812.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_df_stab = new_df_stab.fillna(method='ffill').fillna(method='bfill')


In [9]:
test_df = df.loc[df["_type"] == "test"]
new_test_df = test_df[['ID','target','_type'] + non_missing_columns]

new_test_stab = new_test_df[non_missing_columns]
# test
window_size = 3
new_test_stab = new_test_stab.apply(lambda col: col.fillna(col.rolling(window=window_size, min_periods=1).mean()))
new_test_stab = new_test_stab.fillna(method='ffill').fillna(method='bfill')

new_test_df = pd.concat([new_test_df[['ID','target','_type']], new_test_stab], axis=1)

/tmp/ipykernel_751711/4038964083.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_test_stab = new_test_stab.fillna(method='ffill').fillna(method='bfill')


In [10]:
# 결측치 비율을 계산
missing_percentage = new_test_df.isnull().mean() * 100

# 결측치 비율이 100%인 컬럼 이름만 출력
columns_with_all_missing = missing_percentage[missing_percentage >= 50].index.tolist()

# 100% 결측치가 있는 컬럼 출력
columns_with_all_missing = [col for col in columns_with_all_missing if col not in ['target', 'hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close']]
columns_with_all_missing

['hourly_market-data_open-interest_binance_btc_busd_open_interest',
 'hourly_market-data_liquidations_binance_btc_busd_short_liquidations_usd',
 'hourly_market-data_liquidations_binance_btc_busd_long_liquidations',
 'hourly_market-data_liquidations_binance_btc_busd_short_liquidations',
 'hourly_market-data_liquidations_binance_btc_busd_long_liquidations_usd',
 'hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_volume']

In [11]:
# train_df와 test_df에서 columns_with_all_missing에 있는 컬럼 삭제
new_train_df = new_train_df.drop(columns=columns_with_all_missing, errors='ignore')
new_test_df = new_test_df.drop(columns=columns_with_all_missing, errors='ignore')

In [12]:
# 이동평균을 기반으로 이상치를 처리하는 함수
def replace_outlier(df, window=3, threshold=2):
    df_cleaned = df.copy()
    
    # 숫자형 컬럼들에 대해 처리
    for column in df_cleaned.select_dtypes(include=[np.number]).columns:
        # 이동평균과 표준편차 계산
        rolling_mean = df_cleaned[column].rolling(window=window, min_periods=1).mean()
        rolling_std = df_cleaned[column].rolling(window=window, min_periods=1).std()

        # 이상치 기준 설정
        outliers = np.abs(df_cleaned[column] - rolling_mean) > (threshold * rolling_std)

        # 이상치를 이동평균으로 대체
        df_cleaned.loc[outliers, column] = rolling_mean[outliers]
    
    return df_cleaned

# 이동평균 기반 이상치 처리 적용
cleaned_train_df = replace_outlier(new_train_df)
cleaned_test_df = replace_outlier(new_test_df)

In [35]:
# target과의 연관점수가 높은 150개의 피처 select
from sklearn.feature_selection import f_classif, SelectKBest

n_feature = 150
sel_columns = cleaned_train_df.drop(["target", "ID", "_type"], axis=1).columns.tolist()

selector = SelectKBest(f_classif, k = n_feature)
x_train_sel = selector.fit_transform(cleaned_train_df[sel_columns], cleaned_train_df["target"])
top_features = sorted(zip(sel_columns, selector.scores_), key = lambda x : x[1], reverse = True)
top_col_names = [t[0] for t in top_features[:n_feature]]

print(top_col_names)

['hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume', 'hourly_market-data_taker-buy-sell-stats_binance_taker_sell_volume', 'hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume', 'hourly_market-data_taker-buy-sell-stats_binance_taker_buy_volume', 'hourly_market-data_taker-buy-sell-stats_bybit_taker_sell_volume', 'hourly_market-data_taker-buy-sell-stats_bybit_taker_buy_volume', 'hourly_market-data_taker-buy-sell-stats_okx_taker_sell_volume', 'hourly_market-data_taker-buy-sell-stats_deribit_taker_sell_volume', 'hourly_market-data_taker-buy-sell-stats_okx_taker_buy_volume', 'hourly_market-data_taker-buy-sell-stats_deribit_taker_buy_volume', 'hourly_market-data_taker-buy-sell-stats_bitmex_taker_sell_volume', 'hourly_market-data_taker-buy-sell-stats_bitmex_taker_buy_volume', 'hourly_market-data_taker-buy-sell-stats_huobi_global_taker_buy_volume', 'hourly_market-data_taker-buy-sell-stats_htx_global_taker_buy_volume', 'hourly_market-data_taker-buy-sell-st

In [36]:
sel_train_df = cleaned_train_df[["target", "ID", "_type"]+top_col_names]
sel_test_df = cleaned_test_df[["target", "ID", "_type"]+top_col_names]

print(sel_train_df.shape, sel_test_df.shape)

(8760, 153) (2792, 153)


### 정규화

In [37]:
# Standardization으로 정규화
from sklearn.preprocessing import StandardScaler

def standardization(train_df, test_df):
    features_to_scale = [col for col in train_df.columns if col not in ['ID', 'target', '_type']]

    scaler = StandardScaler()

    # 훈련 데이터 정규화
    train_df_scaled = train_df.copy()
    train_df_scaled[features_to_scale] = scaler.fit_transform(train_df[features_to_scale])

    # 테스트 데이터 정규화
    test_df_scaled = test_df.copy()
    test_df_scaled[features_to_scale] = scaler.transform(test_df[features_to_scale])

    return train_df_scaled, test_df_scaled

std_train_df, std_test_df = standardization(sel_train_df, sel_test_df)


In [67]:
df = pd.concat([std_train_df, std_test_df], ignore_index=True)
df

,target,ID,_type,hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume,hourly_market-data_taker-buy-sell-stats_binance_taker_sell_volume,hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume,hourly_market-data_taker-buy-sell-stats_binance_taker_buy_volume,hourly_market-data_taker-buy-sell-stats_bybit_taker_sell_volume,hourly_market-data_taker-buy-sell-stats_bybit_taker_buy_volume,hourly_market-data_taker-buy-sell-stats_okx_taker_sell_volume,...,hourly_market-data_liquidations_bitmex_btc_usd_short_liquidations_usd,hourly_market-data_liquidations_huobi_global_btc_usd_short_liquidations_usd,hourly_market-data_liquidations_htx_global_btc_usd_short_liquidations_usd,hourly_market-data_open-interest_htx_global_btc_usd_open_interest,hourly_market-data_open-interest_huobi_global_btc_usd_open_interest,hourly_network-data_fees-transaction_fees_transaction_mean,hourly_network-data_fees-transaction_fees_transaction_median_usd,hourly_market-data_taker-buy-sell-stats_huobi_global_taker_buy_ratio,hourly_network-data_fees_fees_total,hourly_market-data_funding-rates_bybit_funding_rates
0,2.0,2023-01-01 00:00:00,train,-0.704407,-0.693055,-0.712152,-0.701412,-0.790786,-0.739005,-0.646295,...,-0.086404,-0.052688,-0.052688,-0.099874,-0.099872,-0.452496,-0.508875,1.712023,-0.472265,0.242277
1,1.0,2023-01-01 01:00:00,train,-0.791420,-0.785440,-0.733461,-0.736673,-0.812900,-0.555248,-0.680815,...,-0.086404,-0.052688,-0.052688,0.005954,0.005957,-0.476448,-0.508594,2.482621,-0.540988,0.242277
2,1.0,2023-01-01 02:00:00,train,-0.793332,-0.790240,-0.783198,-0.782640,-0.783052,-0.844533,-0.737382,...,-0.086404,-0.052688,-0.052688,-0.005201,-0.005198,-0.417162,-0.509145,1.581425,-0.528197,0.242277
3,1.0,2023-01-01 03:00:00,train,-0.763767,-0.756616,-0.768791,-0.766650,-0.841254,-0.809874,-0.695938,...,-0.086404,-0.052688,-0.052688,-0.036679,-0.036676,-0.486115,-0.509212,2.083775,-0.549200,0.242277
4,2.0,2023-01-01 04:00:00,train,-0.766216,-0.767929,-0.761585,-0.757060,-0.660023,-0.701877,-0.699880,...,-0.086404,-0.052688,-0.052688,-0.036056,-0.036054,-0.449115,-0.508374,-0.516556,-0.521007,0.242277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,NaN,2024-04-26 03:00:00,test,-0.055898,-0.396219,0.154741,-0.456799,-0.681878,-0.714295,-0.054989,...,-0.086404,-0.052688,-0.052688,1.979640,1.979640,0.423228,1.226161,-1.846190,-0.071169,0.242277
11548,NaN,2024-04-26 04:00:00,test,-0.055898,-0.263354,0.154741,-0.315382,-0.189639,-0.480559,0.262365,...,-0.086404,-0.052688,-0.052688,1.964823,1.964823,0.517991,0.916341,-1.846190,0.313377,0.242277
11549,NaN,2024-04-26 05:00:00,test,-0.055898,-0.171326,0.154741,0.010637,-0.113936,-0.166396,0.295655,...,-0.086404,-0.052688,-0.052688,1.969874,1.969874,0.155701,0.483112,-1.846190,0.247242,0.242277
11550,NaN,2024-04-26 06:00:00,test,-0.055898,-0.327024,0.154741,-0.465079,-0.603343,-0.578828,0.238835,...,-0.086404,-0.052688,-0.052688,1.944643,1.944643,0.000509,0.314385,-1.846190,-0.160906,0.242277


In [76]:
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_gap": "coinbase_premium_gap",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_index": "coinbase_premium_index",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations_usd": "long_liquidations_usd",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations_usd": "short_liquidations_usd",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_ratio": "buy_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_sell_ratio": "buy_sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume": "buy_volume",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_ratio": "sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume": "sell_volume",
    "hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close" : "close",
}

col_list = list(set(list(cols_dict.keys()) + top_col_names))
df = df[col_list].rename(cols_dict, axis=1)
df

,hourly_market-data_funding-rates_binance_funding_rates,hourly_market-data_liquidations_gate_io_all_symbol_short_liquidations_usd,hourly_market-data_liquidations_huobi_global_btc_usd_long_liquidations,hourly_market-data_taker-buy-sell-stats_okx_taker_buy_sell_ratio,hourly_market-data_open-interest_deribit_all_symbol_open_interest,hourly_market-data_liquidations_htx_global_btc_usdt_long_liquidations_usd,hourly_market-data_liquidations_okx_btc_usdt_long_liquidations_usd,hourly_market-data_liquidations_binance_btc_usd_short_liquidations,hourly_market-data_liquidations_okx_btc_usdt_short_liquidations,hourly_market-data_funding-rates_bybit_funding_rates,...,hourly_market-data_liquidations_huobi_global_all_symbol_long_liquidations_usd,hourly_network-data_tokens-transferred_tokens_transferred_mean,hourly_market-data_open-interest_binance_btc_usdt_open_interest,hourly_market-data_taker-buy-sell-stats_okx_taker_buy_ratio,hourly_market-data_liquidations_htx_global_btc_usd_long_liquidations,hourly_market-data_liquidations_bitfinex_all_symbol_short_liquidations,hourly_market-data_liquidations_huobi_global_all_symbol_long_liquidations,hourly_market-data_liquidations_binance_btc_usd_short_liquidations_usd,ID,hourly_market-data_taker-buy-sell-stats_huobi_global_taker_buy_volume
0,0.379556,-0.189781,-0.096814,-1.039768,-1.679411,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,-0.293700,0.300183,-2.266495,-1.830485,-0.096814,-0.077494,-0.289134,-0.216871,2023-01-01 00:00:00,-0.302190
1,0.379556,-0.189781,-0.096814,-0.448376,-1.677487,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,-0.293700,-0.140147,-2.259519,-0.486496,-0.096814,-0.077494,-0.289134,-0.216871,2023-01-01 01:00:00,0.177370
2,0.379556,-0.189781,-0.096814,5.597865,-1.677390,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,-0.293700,1.601204,-2.257305,3.286031,-0.096814,-0.077494,-0.289134,-0.216871,2023-01-01 02:00:00,-0.308924
3,0.379556,-0.189781,-0.096814,-0.168942,-1.677866,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,-0.293700,2.515394,-2.257566,-0.014928,-0.096814,-0.077494,-0.289134,-0.216871,2023-01-01 03:00:00,0.094990
4,0.379556,-0.189781,-0.096814,0.353864,-1.677846,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,-0.293700,0.919812,-2.255830,0.686127,-0.096814,-0.077494,-0.289134,-0.216871,2023-01-01 04:00:00,-0.570161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,-0.358053,-0.189781,-0.096814,0.030750,3.158508,-0.263834,-0.175608,-0.218655,-0.21572,0.242277,...,-0.116478,0.620859,3.290291,0.276998,-0.096814,-0.077494,-0.212331,-0.216871,2024-04-26 03:00:00,-0.507358
11548,-0.171002,-0.189781,-0.096814,-0.543703,3.188130,0.128419,-0.175305,-0.218655,-0.21572,0.242277,...,-0.116478,0.524770,3.279722,-0.667410,-0.096814,-0.077494,-0.212331,-0.216871,2024-04-26 04:00:00,-0.507358
11549,-0.264528,-0.189781,-0.096814,0.083949,3.205984,-0.188003,-0.175608,-0.218655,-0.21572,0.242277,...,-0.116478,-0.911523,3.298133,0.349360,-0.096814,-0.077494,-0.212331,-0.216871,2024-04-26 05:00:00,-0.507358
11550,-0.171002,-0.189781,-0.096814,-0.562108,3.116695,-0.219970,-0.175608,-0.218655,-0.21572,0.242277,...,-0.116478,0.029348,3.277169,-0.703668,-0.096814,-0.077494,-0.212331,-0.216871,2024-04-26 06:00:00,-0.507358


### Feature Engineering
12시간 기준의 shift feature, 24시간 기준의 rolling feature를 생성한다.

In [77]:
# eda에서 파악한 차이와 음수, 양수 여부 새로운 feature로 생성
df = df.assign(
    liquidation_diff=df["long_liquidations"] - df["short_liquidations"],
    liquidation_usd_diff=df["long_liquidations_usd"] - df["short_liquidations_usd"],
    volume_diff=df["buy_volume"] - df["sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"] - df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(df["long_liquidations_usd"] - df["short_liquidations_usd"]),
    volume_diffg=np.sign(df["buy_volume"] - df["sell_volume"]),
    buy_sell_volume_ratio=df["buy_volume"] / (df["sell_volume"] + 1),
    close_diff = df['close'].diff().fillna(0),
    close_diffg = np.sign(df['close'].diff().fillna(0))
)

# category, continuous 열을 따로 할당해둠
category_cols: List[str] = ["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg", "close_diffg"]
conti_cols: List[str] = [_ for _ in cols_dict.values() if _ not in ["ID", "target", "_type"]] + [
    "buy_sell_volume_ratio",
    "liquidation_diff",
    "liquidation_usd_diff",
    "volume_diff",
    "close_diff"
]

In [78]:
df = df.rename(cols_dict, axis=1)
df.shape

(11552, 162)

In [79]:
from typing import List
import pandas as pd

def rolling_feature(
   df: pd.DataFrame,
   conti_cols: List[str],
   intervals: List[int],
   funcs: List[str],
   min_periods: int,
   **params,
) -> pd.DataFrame:
   """
   Create rolling features
   Args:
       df (pd.DataFrame): Sorted dataframe
       conti_cols (List[str]): continuous colnames
       intervals (List[str]): rolling window widths
       funcs (List[str]): aggregation functions e.g. ["mean", "median", "max"]
       **params: more input for rolling
   Returns:
       pd.DataFrame
   """
   df_rolling_list = [
       df[conti_col]
       .rolling(interval, min_periods= min_periods, **params)
       .agg({f"{conti_col}": func})
       .rename({conti_col: f"{conti_col}_{func}_{interval}"}, axis=1)
       
       
       for conti_col in conti_cols
       for interval in intervals
       for func in funcs
   ]
   return pd.concat(df_rolling_list, axis=1)

In [80]:
# conti_cols : liquidation, taker_volume
# intervals : 24hours
# func : mean()
conti_cols: List[str] = [
    "buy_sell_volume_ratio",
    "liquidation_diff",
    "liquidation_usd_diff",
    "volume_diff",
    "close_diff"
]

intervals = [24]
funcs = ["mean"]

# 초기 데이터는 이전 값이 주어지지 않으므로, min_periods = 1로 설정해 NaN값을 방지한다.
rolling_df = rolling_feature(df, conti_cols, intervals, funcs, 1)
rolling_df

,buy_sell_volume_ratio_mean_24,liquidation_diff_mean_24,liquidation_usd_diff_mean_24,volume_diff_mean_24,close_diff_mean_24
0,-2.409234,-0.008274,0.007011,-0.007745,0.000000
1,-2.962845,-0.012861,0.004051,0.025107,0.001732
2,-3.238447,-0.011381,0.005008,0.020116,0.000646
3,-3.242430,-0.008831,0.006587,0.013831,-0.000132
4,-3.245472,-0.007867,0.007191,0.011991,-0.000409
...,...,...,...,...,...
11547,0.163903,0.059785,0.132607,0.210639,0.000000
11548,0.163903,0.062909,0.140267,0.210639,0.000000
11549,0.163903,0.061005,0.135439,0.210639,0.000000
11550,0.163903,0.060588,0.134351,0.210639,0.000000


shift 피처를 생성한다.

In [81]:
def shift_feature(
    df: pd.DataFrame,
    conti_cols: List[str],
    intervals: List[int],
) -> List[pd.Series]:
    """
    연속형 변수의 shift feature 생성
    Args:
        df (pd.DataFrame)
        conti_cols (List[str]): continuous colnames
        intervals (List[int]): shifted intervals
    Return:
        List[pd.Series]
    """
    df_shift_dict = [
        df[conti_col].shift(interval).rename(f"{conti_col}_{interval}")
        for conti_col in conti_cols
        for interval in intervals
    ]
    return pd.concat(df_shift_dict, axis=1)


# 최대 12시간의 shift 피쳐를 계산
shift_df = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 12)]
)
shift_df

,buy_sell_volume_ratio_1,buy_sell_volume_ratio_2,buy_sell_volume_ratio_3,buy_sell_volume_ratio_4,buy_sell_volume_ratio_5,buy_sell_volume_ratio_6,buy_sell_volume_ratio_7,buy_sell_volume_ratio_8,buy_sell_volume_ratio_9,buy_sell_volume_ratio_10,...,close_diff_2,close_diff_3,close_diff_4,close_diff_5,close_diff_6,close_diff_7,close_diff_8,close_diff_9,close_diff_10,close_diff_11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-2.409234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-3.516455,-2.409234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3.789651,-3.516455,-2.409234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003464,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-3.254379,-3.789651,-3.516455,-2.409234,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.001527,0.003464,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11548,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11549,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11550,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,0.163903,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
# 롤링 피처와 시프트 피처가 추가된 feat_df
feat_df = pd.concat([df, rolling_df, shift_df], axis=1)
feat_df

,hourly_market-data_funding-rates_binance_funding_rates,hourly_market-data_liquidations_gate_io_all_symbol_short_liquidations_usd,hourly_market-data_liquidations_huobi_global_btc_usd_long_liquidations,hourly_market-data_taker-buy-sell-stats_okx_taker_buy_sell_ratio,hourly_market-data_open-interest_deribit_all_symbol_open_interest,hourly_market-data_liquidations_htx_global_btc_usdt_long_liquidations_usd,hourly_market-data_liquidations_okx_btc_usdt_long_liquidations_usd,hourly_market-data_liquidations_binance_btc_usd_short_liquidations,hourly_market-data_liquidations_okx_btc_usdt_short_liquidations,hourly_market-data_funding-rates_bybit_funding_rates,...,close_diff_2,close_diff_3,close_diff_4,close_diff_5,close_diff_6,close_diff_7,close_diff_8,close_diff_9,close_diff_10,close_diff_11
0,0.379556,-0.189781,-0.096814,-1.039768,-1.679411,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.379556,-0.189781,-0.096814,-0.448376,-1.677487,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.379556,-0.189781,-0.096814,5.597865,-1.677390,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.379556,-0.189781,-0.096814,-0.168942,-1.677866,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,0.003464,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.379556,-0.189781,-0.096814,0.353864,-1.677846,-0.307705,-0.175781,-0.218655,-0.21572,0.242277,...,-0.001527,0.003464,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,-0.358053,-0.189781,-0.096814,0.030750,3.158508,-0.263834,-0.175608,-0.218655,-0.21572,0.242277,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11548,-0.171002,-0.189781,-0.096814,-0.543703,3.188130,0.128419,-0.175305,-0.218655,-0.21572,0.242277,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11549,-0.264528,-0.189781,-0.096814,0.083949,3.205984,-0.188003,-0.175608,-0.218655,-0.21572,0.242277,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11550,-0.171002,-0.189781,-0.096814,-0.562108,3.116695,-0.219970,-0.175608,-0.218655,-0.21572,0.242277,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 종가 예측 모델

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def RandomForestReg(feat_df):
    df = feat_df.copy()
    # 타겟과 피처 설정
    y_train = df[df['_type']=='train']['close']
    X_train = df[df['_type']=='train'].drop(columns=['close', 'ID', 'target', '_type'])

    # 훈련 데이터와 검증 데이터 나누기
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # 모델 훈련
    model = RandomForestRegressor()
    model.fit(X_train_split, y_train_split)

    # 검증 데이터에서 예측
    y_val_pred = model.predict(X_val)

    # 성능 평가
    mse = mean_squared_error(y_val, y_val_pred)
    print("Mean Squared Error on Validation Set:", mse)

    # 모델 훈련
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # test_df에서 예측
    X_test = df[df['_type']=='test'].drop(columns=['close', 'ID', 'target', '_type'], errors='ignore')
    y_pred = model.predict(X_test)

    return y_pred

y_pred = RandomForestReg(feat_df)

Mean Squared Error on Validation Set: 0.0008990063484564403


In [94]:
feat_df.loc[feat_df['_type']=='test', 'close']= y_pred
feat_df['close']

0       -2.083215
1       -2.079750
2       -2.081277
3       -2.083744
4       -2.085260
           ...   
11547    2.507502
11548    2.505671
11549    2.509988
11550    2.507942
11551    2.508407
Name: close, Length: 11552, dtype: float64

In [95]:
df = feat_df.copy()

# 타겟 변수를 제외한 변수를 forwardfill, -999로 결측치 대체
_target = df["target"]
df = df.ffill().fillna(-999).assign(target = _target)

# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

In [106]:
# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis = 1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
)

# lgb dataset
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 4,
    "num_leaves": 50,
    "learning_rate": 0.01,
    "n_estimators": 100,
    "random_state": 42,
    "verbose": 0,
}

# lgb train
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=valid_data,
)

# lgb predict
y_valid_pred = lgb_model.predict(x_valid)
y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)

# score check
accuracy = accuracy_score(y_valid, y_valid_pred_class)
auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

print(f"acc: {accuracy}, auroc: {auroc}")

/data/ephemeral/home/myenv/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4594748858447489, auroc: 0.6552039161142038


In [108]:
# performance 체크후 전체 학습 데이터로 다시 재학습
x_train = pd.concat([x_train, x_valid], axis=0)
y_train = pd.concat([y_train, y_valid], axis=0)

train_data = lgb.Dataset(x_train, label=y_train)
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
)

/data/ephemeral/home/myenv/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [109]:
# lgb predict
y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis = 1))
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

In [110]:
# output file 할당후 save 
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df.to_csv("output/output_5.csv", index=False)

-----